In [1]:
import pyprob
from pyprob import Model
from pyprob.distributions import Normal, Uniform, Categorical, Beta

import torch
import numpy as np
import pandas as pd
import math
import random
import matplotlib.pyplot as plt;
import seaborn as sns; sns.set()
%matplotlib inline

In [2]:
faces_to_values = {str(i): i for i in range(2,11)}
faces_to_values['A'] = 1
for f in ['K', 'Q', 'J']:
    faces_to_values[f] = 10

In [3]:
class Card:
    def __init__(self, face):
        """
        Value: {1, ..., 10, 11, 12, 13} - 11 is J, 12 is Q, 13 is K
        """
        self.face = face
        self.value = faces_to_values[face]
    
    def __repr__(self):
        return self.face

In [4]:
Card('K').value

10

In [5]:
def create_deck(num_decks):
    deck = []
    for _ in range(num_decks):
        for face in faces_to_values.keys():
            new_card = Card(face)
            deck.append(new_card)
    
    return deck

In [6]:
deck = create_deck(2)

In [7]:
len(deck)

26

In [8]:
class Blackjack(Model):
    
    def __init__(self, num_decks):
        super().__init__(name='Number game')
        self.num_decks = num_decks
        self.deck = self._create_decks()
        self.cards = {}
    
    def _create_decks(self):
        deck = []
        for _ in range(self.num_decks):
            for face in faces_to_values.keys():
                new_card = Card(face)
                deck.append(new_card)

        return deck
    
    
    def _deal(self, deck):
        np.random.shuffle(deck)
        
        # Dealer
        dealer_card1 = deck.pop()
        dealer_card2 = deck.pop()
        self.cards['dealer'] = [dealer_card1, dealer_card2]
        
        # Player
        player_card1 = deck.pop()
        player_card2 = deck.pop()
        self.cards['player'] = [player_card1, player_card2]
            
    
    def compute_prob_draw_value(self, deck):
        counts = np.zeros(11)
        for card in deck:
            counts[card.value] += 1
        return counts
            
    
    def forward(self):
        """
        Generative sampling process
        """
        score = 0  # latent variable to infer
        
        deck_copy = self.deck.copy()
        self._deal(deck_copy)
        
        hidden_cards = self.cards['dealer']
        
        score += hidden_cards[0].value + hidden_cards[1].value
        
        i = 0
        while score < 16:
            likelihood = Categorical(self.compute_prob_draw_value(self.deck))
            new_card = deck_copy.pop()
            pyprob.observe(likelihood, name='obs{}'.format(i))
            score += new_card.value
            i += 1
        
        return score
    
    
model = Blackjack(num_decks=2)
# model_nn = NumberGame(max_num=100, mult_range_start=2, mult_range_end=10, interval_range_length=10, mult_weight=20)

In [9]:
prior = model.prior_results(num_traces=1000)

Time spent  | Time remain.| Progress             | Trace     | Traces/sec
0d:00:00:00 | 0d:00:00:00 | #################### | 1000/1000 | 4,583.10       


In [86]:
prior.plot_histogram(show=False, alpha=0.75, label='empirical prior', bins=99)
plt.title('Prior for concepts')
plt.xlabel('Concept indices')
# plt.savefig('images/concept_prior.png')

Text(0.5, 0, 'Concept indices')